In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp/scripts_python'

In [1]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import cv2

In [3]:
def safe_mkdir(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

In [4]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [5]:
def save_video(videodata,ofn='outvideo.mp4'):
    # 20FPS, opens in quicktime
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-pix_fmt' : 'yuv420p','-codec': 'h264', '-filter:v' : 'fps=20'})
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=20'})
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8))

In [6]:
def return_frame2(img,enlarge=0):
    
    trained_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #img = cv2.imread(image_fn)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cordinates = trained_data.detectMultiScale(gray_img)
    face_cordinates = face_cordinates + [-enlarge,-enlarge,enlarge*2,enlarge*2]
    
    return face_cordinates

def crop_video_to_rect2(video_data,x,y,h,w):
    return np.array([crop_frame_to_rect2(video_data[i,:,:,:],x, y, w, h) for i in range(video_data.shape[0])])

def crop_frame_to_rect2(img,x,y,h,w):
    return img[np.arange(y,y+h),:,:][:,np.arange(x,x+w),:]

def resize_video(video_data,vid_size_target=(512,512)):
    from PIL import Image, ImageEnhance,ImageStat
    import numpy as np
    
    #vid_size_target = np.array((512,512))
    vid_size_target = np.array(vid_size_target)
    vid_size = np.array((video_data.shape[1],video_data.shape[2]))
    nframes = video_data.shape[0]
    nchannels = video_data.shape[-1]
    video_data_new = np.zeros((nframes,vid_size_target[0],vid_size_target[1],nchannels))
    for f in range(nframes):
        video_data_new[f,:,:,:] = np.array(Image.fromarray(video_data[f,:,:,:]).resize(vid_size_target[-1::-1],Image.ANTIALIAS))
        
    return video_data_new

In [9]:
#indir = '/Volumes/Samsung_T5/disfa/Video_RightCamera_cut'
indir = '/Volumes/Samsung_T5/disfa/Videos_LeftCamera_cut'
#indir = '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/'
files = [os.path.join(indir,file) for file in os.listdir(indir) if all((file.endswith('.mp4'),not file.startswith('.')))]
print(len(files))
files.sort()
files[0:5]

34


['/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN003_comp_n1.mp4',
 '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN003_comp_n2.mp4',
 '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN003_comp_n3.mp4',
 '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN005_comp_n1.mp4',
 '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN005_comp_n2.mp4']

In [10]:
outdir = '/Volumes/Samsung_T5/disfa/Video_LeftCamera_cut_cropped/'
#outdir = '/Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral_cropped/'
safe_mkdir(outdir)

In [11]:
for vid_fn in tqdm(files): 
    ofn = os.path.join(outdir,vid_fn.split('/')[-1])
    if not os.path.exists(ofn):
        try:
            video_data = load_video(vid_fn)

            frame_rects = np.array([return_frame2(video_data[i,:,:,:],enlarge=0)[0] for i in range(video_data.shape[0])])
            mean_frame = frame_rects.mean(axis=0).astype(int)
            x, y, w, h = mean_frame
            video_data_cropped = crop_video_to_rect2(video_data,x,y,h,w)
            video_data_cropped = resize_video(video_data_cropped,vid_size_target=(512,512))

            save_video(video_data_cropped,ofn)
        except:
            print(f'passing on {vid_fn}')

  6%|██▌                                         | 2/34 [00:04<01:02,  1.96s/it]

passing on /Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN003_comp_n2.mp4


  9%|███▉                                        | 3/34 [00:05<00:58,  1.88s/it]

passing on /Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN003_comp_n3.mp4


 12%|█████▏                                      | 4/34 [00:07<00:53,  1.78s/it]

...

passing on /Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN031_comp_n1.mp4


 97%|█████████████████████████████████████████▋ | 33/34 [00:54<00:01,  1.53s/it]

passing on /Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN031_comp_n2.mp4


100%|███████████████████████████████████████████| 34/34 [00:55<00:00,  1.64s/it]

passing on /Users/aidasaglinskas/Desktop/use_expressions_disfa/neutral/RightVideoSN031_comp_n3.mp4
